# Value-based bidding

Value-based bidding is a strategy where advertisers set bids based on the estimated value each ad click brings to their business, aiming to optimize return on investment.

In performance marketing, ad bidding involves setting a price you're willing to pay for specific actions or outcomes from your ads, such as clicks, conversions, or impressions. 

To determine if your ad bidding strategy is profitable and to price your bid optimally for revenue, the primary goal is to ensure that the cost of acquiring a customer (CAC) through ads is less than the customer's lifetime value (LTV) to your business. This is not a faddish metric, and it follows directly from the basic profit equation.

$$ \text{Profitable} \iff LTV_i > CAC_i $$

CAC includes all marketing and sales expenses over a specific period. This means advertising spend, salaries of marketing and sales teams, software costs, creative costs, and more.

In practice, CPA is more commonly used, because it's easy to forget about fixed costs.  While CAC includes a wider range of costs (marketing, sales, overheads), whereas CPA is typically confined to direct campaign costs.

$$ \text{Maybe profitable} \iff LTV_i > CPA_i $$


Return on ad-spend (ROAS) can be thought of as a myopic LTV. ROAS measures the immediate effectiveness of specific advertising campaigns in generating revenue.  ROAS is calculated by dividing the revenue generated from a particular advertising campaign by the cost of that campaign. The time cutoff depends on 'revenue generated' depends on the specific goals and context of the advertising campaign. There is no standard, universally applicable time frame. As a result, it is a vague and static metric. Sometimes, 'liftime ROAS' is used as a synonym for a modified LTV/CPA ratio which takes into account the lifetime revenue generated.

$$\text{Lifetime ROAS} = \dfrac{LTV}{CPA} - 1 $$

# Bidding strategies

Broadly speaking, bidding strategies break into automated and manual.

### Automated bidding strategies

Within automated bidding strategies, we break again into maximisation and targeting.

Maximisation strategies aim to maximize a specific aspect of your campaign, such as clicks or conversions, within your budget.

- **Maximize Clicks:** Focuses on driving as many clicks as possible to your website within your set budget.
- **Maximize Conversions:** Aims to get as many conversions as possible within your specified budget.
- **Maximize Conversion Value:** Seeks to maximize the total conversion value (like revenue) instead of the number of conversions, within your budget.

Targeting strategies are focused on achieving a specific target, such as a desired CPA or ROAS.

- **Target Impression Share:** Sets bids to achieve a specified impression share percentage on the search result page.
- **tCPA (Target Cost Per Acquisition):** Sets bids to target an average CPA that you specify. This strategy tries to get as many conversions as possible at the set target CPA.
- **tROAS (Target Return On Ad Spend):** Aims to achieve a target return on ad spend. It automatically sets bids to maximize conversion value while trying to reach the target ROAS.


### Manual bidding strategies
- **Manual CPC:** Direct control over bids for clicks.
- **eCPC (Enhanced CPC):** Allows some automatic adjustment for clicks likely to lead to sales or conversions.

Automated bidding strategies, specifically Maximise Conversion Value (MCV) and Target ROAS (tROAS) are the closest to a value-based bidding strategy. 
- In MCV, the bidding algorithm may aggressively pursue high-value conversions, even if they come at a higher cost, as long as they contribute to the overall revenue. 
- In tROAS, the algorithm adjusts bids to meet the specific return ratio. This might involve passing up on certain high-cost conversions if they don't meet the ROAS lower bound, potentially leading to different bidding decisions compared to purely maximizing revenue. 

While both strategies aim to maximize revenue, MCV does so without the constraint of a specific return ratio, whereas tROAS seeks to balance revenue maximization with a lower bound on ROAS. The distinction becomes more pronounced based on how the target ROAS is set relative to the maximum achievable ROAS.

### Best-in-class solution

We define value using CLV, the present value of revenue flows:

$$\text{CLV}_i = \frac{P_i \times V_i \times r_i}{1 + \text{WACC} - r_i}$$

- $P_i$ is price per seat
- $V_i$ is seat volume
- $r_i$ is retention rate
- $WACC$ is the weighted-average cost of capital

We need to predict CLV for each customer in the first 24-hours of sign up so we can send that information to advertisers.

Decomposing the uncertainty of $CLV$, the major assumption is that the majority of entropy is in whether the seat volume is > 0 or not. That is, whether or not people convert is the biggest unknown.

**Tl;dr:**
1. Calculate CLV using known information for current signups.
2. Train regression model to predict CLV on unseen signups.

**Calculating CLV using known information**
- Revenue ($P_i \cdot V_i$): current MRR
- $r_i$: A function of package, with each package estimated using an exponential survival model
- $WACC$: 15%

### Good-enough solution

We still define value as CLV, but we focus on predicting which product the person will convert to (if any).


**Tl;dr:**
1. Calculate CLV using known information for current signups.
2. Train classification model to predict land package on unseen signups.
3. Assign the mean CLV of current customers on that package to that package.


# Data

**$y$ variable**
- $CLV_i$
- $Product_j$

**$x$ variables**
- Segment [str]
- Education email flag [str]
- Company email flag [str]
- Industry [str]
- Revenue [str]
- Employees [str]
- City [str]
- State [str]
- Country [str]
- Data residency [str]
- GA signup flag [str]
- FB signup flag [str]
- Feature counts in first 1 day [int]

Plus first party data
- Checklist user completed % [production.onboarding_checklist_completed: user_id, workspace_id] (what proportion of users have completed checklist)
- Step user conversion % [production.new_user_global_onboarding: step, workspace_id, user_id] (what proportion of users have onboarded)
- Intent mix [production.onboarding_user_intent_selected: intent, workspace_id, user_id] (mix of users' intent with the product)
- Data mix [production.onboarding_import_option_selected: option, workspace_id, user_id] (mix of users' readiness to import data)
- Use case mix [production.onboarding_use_case_selected: use_case, workspace_id, user_id] (mix of users' job roles)

In [ ]:
WITH workspaces as (
    SELECT workspace_id, 
        case
    	    when products_latest ilike '%Starter%' then 'Starter'
    	    when products_latest ilike '%Pro%' then 'Starter'
    	    when products_latest ilike '%Analysis%' then 'Starter'
    	    when products_latest ilike '%Team%' then 'Team'
    	    when products_latest ilike '%Business%' then 'Business'
    	    when products_latest ilike '%Enterprise%' then 'Enterprise'
    	    else 'No conversion' end as product,
       COALESCE(clv, 0) AS clv, 
       COALESCE(workspace_segment, 'Unknown') AS segment,
       CASE 
           WHEN company_domain_education_flag IS NULL THEN 'Unknown' 
           ELSE CASE company_domain_education_flag 
                WHEN TRUE THEN '1' 
                ELSE '0' 
                END 
       END AS education_flag,
       CASE 
           WHEN company_email_provider IS NULL THEN 'Unknown' 
           ELSE CASE company_email_provider 
                WHEN TRUE THEN '1' 
                ELSE '0' 
                END 
       END AS company_email_flag,
       COALESCE(company_industry_group, 'Unknown') AS industry,
       COALESCE(company_estimated_annual_revenue, 'Unknown') AS revenue,
       COALESCE(company_employees_range, 'Unknown') AS employees_range,
       COALESCE(company_city, 'Unknown') AS city,
       COALESCE(company_state, 'Unknown') AS state,
       COALESCE(company_country_code, 'Unknown') AS country_code,
       CASE 
           WHEN ga_signup_flag IS NULL THEN 'Unknown' 
           ELSE CASE ga_signup_flag 
                WHEN TRUE THEN '1' 
                ELSE '0' 
                END 
       END AS ga_signup_flag,
       CASE 
           WHEN fb_signup_flag IS NULL THEN 'Unknown' 
           ELSE CASE fb_signup_flag 
                WHEN TRUE THEN '1' 
                ELSE '0' 
                END 
       END AS fb_signup_flag,
       COALESCE(data_residency_region, 'Unknown') AS residency_region,
       COALESCE(project_count_f1d, 0) AS project_count,
       COALESCE(transcription_count_f1d, 0) AS transcription_count,
       COALESCE(highlight_count_f1d, 0) AS highlight_count,
       COALESCE(tag_count_f1d, 0) AS tag_count,
       COALESCE(insight_count_f1d, 0) AS insight_count,
       COALESCE(reel_created_count_f1d, 0) AS reel_created_count,
       COALESCE(invite_count_f1d, 0) AS invite_count,
       COALESCE(shared_object_note_count_f1d, 0) AS shared_object_note_count,
       COALESCE(shared_object_insight_count_f1d, 0) AS shared_object_insight_count,
       COALESCE(note_viewed_user_count_f1d, 0) AS note_viewed_user_count,
       COALESCE(tag_viewed_user_count_f1d, 0) AS tag_viewed_user_count
    FROM "dbt_staging"."stg_workspace"
    where workspace_created_at >= '2023-01-01'
),

users as (
SELECT 
    u.workspace_id, 
    SUM(CASE 
            WHEN created_at <= workspace_created_at + INTERVAL '1 day' THEN 1
            ELSE 0 
        END) as f1d_users
FROM 
    "dbt_staging"."stg_user" u
LEFT JOIN 
    "dbt_staging"."stg_workspace" w ON u.workspace_id = w.workspace_id
GROUP BY 
    u.workspace_id
),

checklist_completed as (
SELECT 
    oc.context_group_id as workspace_id, 
    oc.timestamp, 
    COUNT(DISTINCT oc.user_id) as onboarded_users
FROM 
    "production"."onboarding_checklist_completed" oc
JOIN 
    "dbt_staging"."stg_workspace" w ON oc.context_group_id = w.workspace_id
WHERE 
    oc.timestamp <= w.workspace_created_at + INTERVAL '1 day'
GROUP BY 
    oc.context_group_id, oc.timestamp

),

checklist_steps as (
    SELECT 
        context_group_id as workspace_id,
        timestamp::date as timestamp,
        COUNT(DISTINCT CASE WHEN step = 'viewed_intro' THEN user_id ELSE NULL END) as viewed_intro,
        COUNT(DISTINCT CASE WHEN step = 'set_role' THEN user_id ELSE NULL END) as set_role,
        COUNT(DISTINCT CASE WHEN step = 'set_avatar' THEN user_id ELSE NULL END) as set_avatar,
        COUNT(DISTINCT CASE WHEN step = 'invited_user' THEN user_id ELSE NULL END) as invited_user
    FROM 
    production.new_user_global_onboarding
    
    GROUP BY 
    workspace_id, timestamp

),

intent as (
SELECT 
    p.context_group_id as workspace_id,
    p.timestamp::date as timestamp, 
    COUNT(DISTINCT CASE WHEN p.intent = 'VideoTranscription' THEN p.user_id ELSE NULL END) as intent_video_transcription,
    COUNT(DISTINCT CASE WHEN p.intent = 'InsightsHub' THEN p.user_id ELSE NULL END) as intent_insights_hub,
    COUNT(DISTINCT CASE WHEN p.intent = 'ResearchAnalysis' THEN p.user_id ELSE NULL END) as intent_research_analysis
FROM 
    production.onboarding_user_intent_selected p
JOIN 
    dbt_staging.stg_workspace w ON p.context_group_id = w.workspace_id
WHERE 
    p.timestamp <= w.workspace_created_at + INTERVAL '1 day'
GROUP BY  1,2

),

usecase as (
SELECT 
    u.context_group_id as workspace_id,
    u.timestamp::date as timestamp, 
    COUNT(DISTINCT CASE WHEN u.use_case = 'SALES' THEN u.user_id ELSE NULL END) as usecase_sales,
    COUNT(DISTINCT CASE WHEN u.use_case = 'MARKETING' THEN u.user_id ELSE NULL END) as usecase_marketing,
    COUNT(DISTINCT CASE WHEN u.use_case = 'PRODUCT_MANAGEMENT' THEN u.user_id ELSE NULL END) as usecase_product_management,
    COUNT(DISTINCT CASE WHEN u.use_case = 'ENGINEERING' THEN u.user_id ELSE NULL END) as usecase_engineering,
    COUNT(DISTINCT CASE WHEN u.use_case = 'OTHER' THEN u.user_id ELSE NULL END) as usecase_other,
    COUNT(DISTINCT CASE WHEN u.use_case = 'MANAGEMENT' THEN u.user_id ELSE NULL END) as usecase_management,
    COUNT(DISTINCT CASE WHEN u.use_case = 'DESIGN' THEN u.user_id ELSE NULL END) as usecase_design,
    COUNT(DISTINCT CASE WHEN u.use_case = 'SUPPORT' THEN u.user_id ELSE NULL END) as usecase_support,
    COUNT(DISTINCT CASE WHEN u.use_case = 'CUSTOMER_SUCCESS' THEN u.user_id ELSE NULL END) as usecase_customer_success,
    COUNT(DISTINCT CASE WHEN u.use_case = 'RESEARCH' THEN u.user_id ELSE NULL END) as usecase_research,
    COUNT(DISTINCT CASE WHEN u.use_case = 'OPERATIONS' THEN u.user_id ELSE NULL END) as usecase_operations,
    COUNT(DISTINCT CASE WHEN u.use_case = 'LEGAL' THEN u.user_id ELSE NULL END) as usecase_legal,
    COUNT(DISTINCT CASE WHEN u.use_case = 'FINANCE' THEN u.user_id ELSE NULL END) as usecase_finance
FROM 
    production.onboarding_use_case_selected u
JOIN 
    dbt_staging.stg_workspace w ON u.context_group_id = w.workspace_id
WHERE 
    u.timestamp <= w.workspace_created_at + INTERVAL '1 day'
GROUP BY 
1,2

),

data_upload as (
SELECT 
    o.context_group_id as workspace_id,
    o.timestamp::date as timestamp, 
    COUNT(DISTINCT CASE WHEN o.option = 'NoData' THEN o.user_id ELSE NULL END) as upload_nodata,
    COUNT(DISTINCT CASE WHEN o.option = 'Data' THEN o.user_id ELSE NULL END) as upload_data
FROM 
    production.onboarding_import_option_selected o
JOIN 
    dbt_staging.stg_workspace w ON o.context_group_id = w.workspace_id
WHERE 
    o.timestamp <= w.workspace_created_at + INTERVAL '1 day'
GROUP BY 1,2
)

select *
from workspaces w
left join users on w.workspace_id = users.workspace_id
left join checklist_completed on w.workspace_id = checklist_completed.workspace_id
left join checklist_steps on w.workspace_id = checklist_steps.workspace_id
left join intent on w.workspace_id = intent.workspace_id
left join usecase on w.workspace_id = usecase.workspace_id
left join data_upload on w.workspace_id = data_upload.workspace_id


# Import

### Import local XLSX file

In [1]:
def import_data():
    import pandas as pd
    file_path = '/Users/patricksweeney/growth/01_Acquisition/03_Value-based bidding/VBB Train 3.xlsx'
    data = pd.read_excel(file_path)
    return data

data = import_data()
data.head()

/Users/patricksweeney/anaconda3/envs/EconML/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,workspace_id,product,clv,segment,education_flag,company_email_flag,industry,revenue,employees_range,city,...,usecase_support,usecase_customer_success,usecase_research,usecase_operations,usecase_legal,usecase_finance,workspace_id.6,timestamp.4,upload_nodata,upload_data
0,00095959-e65b-4256-aeba-06464ae106ac,No conversion,0.0,OTHER,0,0,Diversified Consumer Services,$50M-$100M,251-1K,Utrecht,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
1,000b6e42-2fbd-45c0-9a73-de2cef1ece0d,No conversion,0.0,OTHER,0,0,Diversified Consumer Services,$1M-$10M,11-50,Mumbai,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
2,000cdc27-c036-4702-9ceb-84ea9f806c3d,No conversion,0.0,OTHER,0,0,Diversified Consumer Services,$100M-$250M,251-1K,Clinton,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
3,00190170-ab11-4a39-9a14-075e5d3c68ad,No conversion,0.0,FREE_EMAIL,0,1,Unknown,Unknown,Unknown,Unknown,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
4,001aa956-d9a4-46f0-a528-611936ff34fe,No conversion,0.0,OTHER,1,0,Diversified Consumer Services,$1B-$10B,1K-5K,Unknown,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN


### Check data

In [4]:
def find_missing_values(data):
    total_rows = len(data)
    missing_values = data.isnull().sum()
    missing_percentage = (missing_values / total_rows) * 100

    print("Features with missing values are:")
    for feature, missing_count in missing_values.items():
        percent = missing_percentage[feature]
        print(f"{feature}: Missing Count = {missing_count}, Missing Percentage = {percent:.2f}%")

# Usage
find_missing_values(data)

Features with missing values are:
workspace_id: Missing Count = 0, Missing Percentage = 0.00%
product: Missing Count = 0, Missing Percentage = 0.00%
clv: Missing Count = 0, Missing Percentage = 0.00%
segment: Missing Count = 0, Missing Percentage = 0.00%
education_flag: Missing Count = 0, Missing Percentage = 0.00%
company_email_flag: Missing Count = 0, Missing Percentage = 0.00%
industry: Missing Count = 0, Missing Percentage = 0.00%
revenue: Missing Count = 0, Missing Percentage = 0.00%
employees_range: Missing Count = 0, Missing Percentage = 0.00%
city: Missing Count = 0, Missing Percentage = 0.00%
state: Missing Count = 0, Missing Percentage = 0.00%
country_code: Missing Count = 0, Missing Percentage = 0.00%
ga_signup_flag: Missing Count = 0, Missing Percentage = 0.00%
fb_signup_flag: Missing Count = 0, Missing Percentage = 0.00%
residency_region: Missing Count = 0, Missing Percentage = 0.00%
project_count: Missing Count = 0, Missing Percentage = 0.00%
transcription_count: Missing 

### Replace NaN with zero


In [8]:
import pandas as pd
import numpy as np

def fill_missing(data):
    # Remove all date columns
    data = data.select_dtypes(exclude=['datetime'])

    # Remove columns containing 'workspace_id' that are not exactly 'workspace_id'
    data = data[[col for col in data.columns if col == 'workspace_id' or 'workspace_id' not in col]]

    # Find columns with missing values
    missing_columns = data.columns[data.isnull().any()]

    # Process each column with missing values
    for column in missing_columns:
        # If numeric columns, fill in the missing value / NaN as 0
        if np.issubdtype(data[column].dtype, np.number):
            data[column].fillna(0, inplace=True)
        # If categorical columns (strings etc) fill it in as 'Unknown'
        else:
            data[column].fillna('Unknown', inplace=True)

    return data

data = fill_missing(data)
data.head()

,workspace_id,product,clv,segment,education_flag,company_email_flag,industry,revenue,employees_range,city,...,usecase_management,usecase_design,usecase_support,usecase_customer_success,usecase_research,usecase_operations,usecase_legal,usecase_finance,upload_nodata,upload_data
0,00095959-e65b-4256-aeba-06464ae106ac,No conversion,0.0,OTHER,0,0,Diversified Consumer Services,$50M-$100M,251-1K,Utrecht,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,000b6e42-2fbd-45c0-9a73-de2cef1ece0d,No conversion,0.0,OTHER,0,0,Diversified Consumer Services,$1M-$10M,11-50,Mumbai,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,000cdc27-c036-4702-9ceb-84ea9f806c3d,No conversion,0.0,OTHER,0,0,Diversified Consumer Services,$100M-$250M,251-1K,Clinton,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,00190170-ab11-4a39-9a14-075e5d3c68ad,No conversion,0.0,FREE_EMAIL,0,1,Unknown,Unknown,Unknown,Unknown,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,001aa956-d9a4-46f0-a528-611936ff34fe,No conversion,0.0,OTHER,1,0,Diversified Consumer Services,$1B-$10B,1K-5K,Unknown,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Feature engineering

### One hot encode


One-hot encoding converts categorical variables into a form that can be provided to machine learning algorithms to improve prediction accuracy. It creates binary columns for each category and avoids the misleading ordinal relationships that numeric encoding might imply.

In [9]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

def one_hot_encode(data, exclude):
    # Ensure that 'data' is a pandas DataFrame
    if not isinstance(data, pd.DataFrame):
        raise TypeError("Input data must be a pandas DataFrame.")

    # Validate 'exclude' as a list
    if not isinstance(exclude, list):
        raise TypeError("'exclude' must be a list of columns.")

    # Select string and categorical columns to encode, excluding the specified columns
    columns_to_encode = data.select_dtypes(include=['object', 'category']).columns
    columns_to_encode = [col for col in columns_to_encode if col not in exclude]

    # Apply OneHotEncoder
    encoder = OneHotEncoder(sparse=False, drop='if_binary')
    encoded_data = pd.DataFrame(encoder.fit_transform(data[columns_to_encode]))

    # Fix column names after encoding
    encoded_data.columns = encoder.get_feature_names_out(columns_to_encode)

    # Drop original columns and concatenate encoded data
    data = data.drop(columns_to_encode, axis=1)
    data = pd.concat([data, encoded_data], axis=1)

    return data

data = one_hot_encode(data, ['workspace_id', 'product'])
data.head()

,workspace_id,product,clv,education_flag,ga_signup_flag,fb_signup_flag,project_count,transcription_count,highlight_count,tag_count,...,country_code_VC,country_code_VE,country_code_VI,country_code_VN,country_code_ZA,country_code_ZW,residency_region_Unknown,residency_region_eu-a,residency_region_us-a,residency_region_us-b
0,00095959-e65b-4256-aeba-06464ae106ac,No conversion,0.0,0,0,0,0,4,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,000b6e42-2fbd-45c0-9a73-de2cef1ece0d,No conversion,0.0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,000cdc27-c036-4702-9ceb-84ea9f806c3d,No conversion,0.0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,00190170-ab11-4a39-9a14-075e5d3c68ad,No conversion,0.0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,001aa956-d9a4-46f0-a528-611936ff34fe,No conversion,0.0,1,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


### Best-case performance

In [10]:
def best_case(data, y_variable, exclude, continuous_y=True):
    # Importing necessary libraries
    import pandas as pd
    import numpy as np
    from scipy.stats import entropy, differential_entropy
    from sklearn.feature_selection import mutual_info_regression, mutual_info_classif

    # Exclude specified variables and separate X and Y
    X = data.drop(columns=[y_variable] + exclude)
    Y = data[y_variable]

    # Calculate the entropy of Y-variable
    if continuous_y:
        # Use differential_entropy for continuous Y
        y_entropy = differential_entropy(Y)
        # Calculate mutual information for continuous Y
        mi = mutual_info_regression(X, Y)
    else:
        # Use entropy for discrete Y
        value_counts = Y.value_counts()
        y_entropy = entropy(value_counts)
        # Calculate mutual information for discrete Y
        mi = mutual_info_classif(X, Y)

    total_mi = np.sum(mi)

    # Proportion of uncertainty reduced
    proportion_reduced = total_mi / y_entropy if y_entropy > 0 else 0

    # Print results
    print(f"Entropy of Y-variable: {y_entropy}")
    print(f"Total Mutual Information (excluding interaction): {total_mi}")
    print(f"Proportion of Uncertainty Reduced: {proportion_reduced}")

# Example usage
best_case(data, 'product', ['clv', 'workspace_id'], continuous_y=False)


/Users/patricksweeney/anaconda3/envs/EconML/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/patricksweeney/anaconda3/envs/EconML/lib/python3.11/site-packages/sklearn/utils/validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/patricksweeney/anaconda3/envs/EconML/lib/python3.11/site-packages/sklearn/utils/validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


Entropy of Y-variable: 0.1713146136677787
Total Mutual Information (excluding interaction): 1.1715657022045578
Proportion of Uncertainty Reduced: 6.838679299575182


### Remove redundant features

In [11]:
import pandas as pd
from sklearn.feature_selection import SelectPercentile, mutual_info_regression, mutual_info_classif

def scikit_prune_features(data, y_variable, exclude, percentile):
    # Ensure that 'exclude' is a list
    if not isinstance(exclude, list):
        raise TypeError("'exclude' must be a list of columns.")

    # Separate the features and the target variable
    X = data.drop(columns=[y_variable] + exclude)
    y = data[y_variable]

    # Determine the score function based on the target variable type
    if y.dtype == 'float':
        score_func = mutual_info_regression
    else:
        score_func = mutual_info_classif

    # Apply SelectPercentile
    selector = SelectPercentile(score_func=score_func, percentile=percentile)
    X_new = selector.fit_transform(X, y)

    # Get the selected feature names
    selected_features = X.columns[selector.get_support()]

    # Combine selected features with excluded columns and target variable
    final_data = pd.concat([data[exclude], data[selected_features], data[[y_variable]]], axis=1)

    return final_data

# Example usage:
data = scikit_prune_features(data, 'product', ['workspace_id', 'clv'], 20)
data.head()


/Users/patricksweeney/anaconda3/envs/EconML/lib/python3.11/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/patricksweeney/anaconda3/envs/EconML/lib/python3.11/site-packages/sklearn/utils/validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/patricksweeney/anaconda3/envs/EconML/lib/python3.11/site-packages/sklearn/utils/validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


# Predicting $CLV_i$ in first 24-hours (not being used)

Performance here is terrible: 0% $r^2$

### Regression with gradient boosting


Gradient Boosting is a machine learning technique that builds models sequentially, with each new model correcting the errors of the previous ones, optimizing for a loss function. This approach combines weak predictive models, typically decision trees, into a stronger ensemble, offering high accuracy in various tasks.

In [ ]:
def gradient_boosting_regression(data, y_variable, random_state=42):
    from sklearn.ensemble import GradientBoostingRegressor
    from sklearn.model_selection import train_test_split, cross_val_score, KFold
    from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
    import matplotlib.pyplot as plt
    import numpy as np
    import pandas as pd
    import math
    from scipy.interpolate import UnivariateSpline

    # Separate the features and target variable
    X = data.drop(columns=[y_variable]).select_dtypes(include=np.number)
    y = data[y_variable]

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)

    # 1. Train model
    model = GradientBoostingRegressor(random_state=random_state)
    model.fit(X_train, y_train)

    # 2. Test model
    cv = KFold(n_splits=10, shuffle=True, random_state=random_state)
    cross_val_scores = cross_val_score(model, X, y, cv=cv, scoring='r2')
    
    # Rounding the scores to two decimal places
    rounded_scores = [round(score, 2) for score in cross_val_scores]
    print("Cross-validation scores (R2):", rounded_scores)

    # Predictions
    y_pred = model.predict(X_test)

    # 3. Performance Metrics
    mse = round(mean_squared_error(y_test, y_pred), 2)
    r2 = round(r2_score(y_test, y_pred), 2)
    mae = round(mean_absolute_error(y_test, y_pred), 2)
    print(f"Mean Squared Error: {mse}")
    print(f"R2 Score: {r2}")
    print(f"Mean Absolute Error: {mae}")

    # 4. Plot Predictions vs Actual with adjusted log10 scale
    offset = 1e-6  # Small constant to offset zero or negative values
    adjusted_y_test = np.log10(y_test + offset)
    adjusted_y_pred = np.log10(y_pred + offset)

    plt.scatter(adjusted_y_test, adjusted_y_pred)
    plt.xlabel("Actual Values (log10 scale)")
    plt.ylabel("Predicted Values (log10 scale)")
    plt.title("Predicted vs Actual Values (log10 scale)")
    
    # Line for perfect predictions
    min_val = min(adjusted_y_test.min(), adjusted_y_pred.min())
    max_val = max(adjusted_y_test.max(), adjusted_y_pred.max())
    plt.plot([min_val, max_val], [min_val, max_val], 'k--')

    plt.show()
    
    
    # 5. Feature Importance - Updated to show only top 10
    feature_importance = model.feature_importances_
    sorted_idx = np.argsort(feature_importance)[-10:]  # Get the indices of the top 10 features
    
    plt.barh(X.columns[sorted_idx], feature_importance[sorted_idx])
    plt.xlabel("Gradient Boosting Feature Importance")
    plt.title("Top 10 Features")
    plt.show()

    plt.tight_layout()
    plt.show()

    return model

model = gradient_boosting_regression(data, 'clv')

# Predicting $Package_i$ in first 24 hours (being used)

We need to be minimising our false negative rate, even if we have an overly sensitive classifier.

### Optional: Make product a binary conversion event

In [6]:
import pandas as pd

def make_binary_conversion(data, y_variable):
    # Check if y_variable exists in the dataframe
    if y_variable not in data.columns:
        raise ValueError(f"{y_variable} is not a column in the provided dataframe.")

    # Convert the y_variable to binary
    data[y_variable] = data[y_variable].apply(lambda x: 'No conversion' if x == 'No conversion' else 'Conversion')

    return data

data = make_binary_conversion(data, 'product')
data.head()


,workspace_id,product,clv,segment,education_flag,company_email_flag,industry,revenue,employees_range,city,...,transcription_count,highlight_count,tag_count,insight_count,reel_created_count,invite_count,shared_object_note_count,shared_object_insight_count,note_viewed_user_count,tag_viewed_user_count
0,00095959-e65b-4256-aeba-06464ae106ac,No conversion,0.0,OTHER,0,0,Diversified Consumer Services,$50M-$100M,251-1K,Utrecht,...,4,0,0,0,0,0,0,0,0,0
1,00786b99-40f5-4703-a772-3026df9827ff,No conversion,0.0,FREE_EMAIL,0,1,Software & Services,$10B+,100K+,Mountain View,...,0,0,0,0,0,0,0,0,0,0
2,00ddc9a5-85c6-44d9-9968-c37cdad31fcc,No conversion,0.0,FREE_EMAIL,0,1,Unknown,Unknown,Unknown,Unknown,...,0,0,0,0,0,0,0,0,0,0
3,0160b311-e4f8-4bbd-a06f-e2b4c80d40a1,No conversion,0.0,FREE_EMAIL,0,1,Unknown,Unknown,Unknown,Unknown,...,0,0,0,0,0,0,0,0,1,0
4,0172345b-7159-4c99-88d9-3e4fa521f14d,No conversion,0.0,FREE_EMAIL,0,1,Unknown,Unknown,Unknown,Unknown,...,0,0,0,0,0,0,0,0,0,0


### Classification with gradient boosting

In [7]:
def gradient_boosting(data, y_variable, exclude):
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
    from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, precision_recall_curve, cohen_kappa_score
    import matplotlib.pyplot as plt
    import numpy as np
    import pandas as pd

    random_state = 2
    
    # Ensure 'exclude' is a list
    if not isinstance(exclude, list):
        raise TypeError("'exclude' must be a list of columns.")

    # Separate features and target variable
    X = data.drop(columns=[y_variable] + exclude).select_dtypes(include=np.number)
    y = data[y_variable]

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)

    # Train model
    model = GradientBoostingClassifier()
    model.fit(X_train, y_train)

    # Test model
    cv = StratifiedKFold(n_splits=10)
    cross_val_scores = cross_val_score(model, X, y, cv=cv, scoring='f1_macro')
    print("Cross-validation scores:", cross_val_scores)

    # Predictions
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)

    # 4. Classification report
    print(classification_report(y_test, y_pred))

    # Confusion matrix
    print("Confusion matrix:")
    print(confusion_matrix(y_test, y_pred))

    # Compute ROC, AUC, Precision-Recall for each class
    classes = np.unique(y)
    for i, cls in enumerate(classes):
        fpr, tpr, _ = roc_curve((y_test == cls).astype(int), y_proba[:, i])
        roc_auc = auc(fpr, tpr)
        precision, recall, _ = precision_recall_curve((y_test == cls).astype(int), y_proba[:, i])
        
        # ROC Curve
        plt.figure()
        plt.plot(fpr, tpr, label='Class %s AUC = %0.2f' % (cls, roc_auc))
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('ROC for class %s' % cls)
        plt.legend(loc="lower right")
        plt.show()

        # Precision-Recall Curve
        plt.figure()
        plt.plot(recall, precision)
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.title('Precision-Recall Curve for class %s' % cls)
        plt.show()
    

    # # Compute and plot Lift Chart
    # df_lift = pd.DataFrame({'y_test': y_test, 'y_proba': y_proba})
    # df_lift = df_lift.sort_values(by='y_proba', ascending=False)
    # df_lift['decile'] = pd.qcut(df_lift['y_proba'], 10, labels=False)
    # df_lift['num_positive'] = df_lift['y_test'].cumsum()
    # df_lift['total'] = df_lift.index + 1
    # df_lift['lift'] = df_lift['num_positive'] / df_lift['total']

    
    # Cross-validation score
    print("Average cross-validation score:", np.mean(cross_val_scores))

    # Cohen's Kappa
    kappa = cohen_kappa_score(y_test, y_pred)
    print("Cohen's Kappa:", kappa)

    plt.tight_layout()
    plt.show()
    
    return model

In [ ]:
model = gradient_boosting(data, 'product', ['clv'])

# Get value-weightings

In [ ]:
def get_predictions(data, model, y_variable, exclude):
    import pandas as pd
    import numpy as np

    # Filter data where the target variable equals 0
    target_data = data

    # Exclude specified variables, separate the features, and retain the index
    X_target = target_data.drop(columns=[y_variable] + exclude).select_dtypes(include=np.number)

    # Make predictions
    predictions = model.predict(X_target)
    probabilities = model.predict_proba(X_target)[:, 1]

    # Append predictions and probabilities to the original data
    data.loc[target_data.index, 'Prediction'] = predictions
    data.loc[target_data.index, 'Probability'] = probabilities

    return data

# Example usage
predictions = get_predictions(data, model, 'product', ['clv'])


In [ ]:
def get_weightings(data, prediction_column, clv_column):
    import pandas as pd

    # Calculate the average CLV for each prediction category
    average_clvs = data.groupby(prediction_column)[clv_column].mean()

    # Create a new column 'weighting' with the average CLV for each prediction
    data['weighting'] = data[prediction_column].map(average_clvs)

    # Select only the necessary columns
    data = data[['workspace_id', prediction_column, 'weighting']]

    return data

weightings = get_weightings(data, 'Prediction', 'clv')
weightings.head()


# Save predictions

In [ ]:
def save_predictions(predictions, filename):
    import pandas as pd

    # Ensure the filename ends with '.xlsx'
    if not filename.endswith('.xlsx'):
        filename += '.xlsx'

    # Save to Excel
    predictions.to_excel(filename, index=False)

save_predictions(weightings, 'predictions_vbb.xlsx')

In [ ]:
# Save model